In [2]:
from google.cloud import bigquery
from google.oauth2 import service_account
from pandas.io import gbq
import pandas_gbq

In [3]:
# These first two values will be different on your machine. 
service_path = "/Users/liamhettinger/Documents/Fall_MSBA/Applied_Data_Analytics/wedge/"
service_file = 'fleet-geode-327423-3bd2c958746e.json' # change this to your authentication information  
gbq_proj_id = 'fleet-geode-327423' # change this to your poroject. 
gbq_dataset_id = 'Wedge'

# And this should stay the same. 
private_key = service_path + service_file

In [4]:
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

In [5]:
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [29]:
sql = """
SELECT DISTINCT card_no
FROM `fleet-geode-327423.Wedge.transArchive_*` 
where NOT card_no = 3 
"""
owner_list = pandas_gbq.read_gbq(sql, project_id=gbq_proj_id)
pandas_gbq.to_gbq(owner_list, 'Wedge.Owner_list', project_id=gbq_proj_id, if_exists='replace')

Downloading: 100%|██████████| 27208/27208 [00:01<00:00, 21644.96rows/s]
1it [00:08,  8.06s/it]


In [30]:
owner_list

,card_no
0,42215.0
1,47401.0
2,44939.0
3,44340.0
4,44800.0
...,...
27203,52095.0
27204,52106.0
27205,52614.0
27206,48711.0


In [6]:
sql = """
SELECT DISTINCT card_no
FROM `fleet-geode-327423.Wedge.transArchive_*` TABLESAMPLE SYSTEM (.01 PERCENT)
where NOT card_no = 3 
"""
Sample_owner_list = pandas_gbq.read_gbq(sql, project_id=gbq_proj_id)
pandas_gbq.to_gbq(Sample_owner_list, 'Wedge.Sample_Owner_list', project_id=gbq_proj_id, if_exists='replace')

Downloading: 100%|██████████| 7432/7432 [00:00<00:00, 10911.97rows/s]
1it [00:02,  2.98s/it]


In [9]:
sql = """
    SELECT *
    FROM `fleet-geode-327423.Wedge.transArchive_*`
    WHERE card_no IN
    (SELECT card_no 
     FROM `fleet-geode-327423.Wedge.Sample_Owner_list`)
"""
All_data_sample_owners = pandas_gbq.read_gbq(sql, project_id=gbq_proj_id)
pandas_gbq.to_gbq(All_data_sample_owners, 'Wedge.All_data_sample_owners', project_id=gbq_proj_id, if_exists='replace')



Downloading:   0%|          | 0/42438509 [00:00<?, ?rows/s]

Downloading:  35%|███▌      | 14871821/42438509 [18:16:05<33:51:43, 226.14rows/s]


Downloading:   0%|          | 21355/42438509 [00:26<4:47:49, 2456.17rows/s]

Downloading:   0%|          | 42719/42438509 [00:28<6:38:27, 1773.30rows/s]

Downloading:   0%|          | 63634/42438509 [00:35<5:47:33, 2032.03rows/s]

Downloading:   0%|          | 84678/42438509 [00:42<5:12:05, 2261.83rows/s]

Downloading:   0%|          | 105804/42438509 [00:47<4:31:16, 2600.89rows/s]

Downloading:   0%|          | 126877/42438509 [00:53<4:13:19, 2783.70rows/s]

Downloading:   0%|          | 147689/42438509 [01:00<4:02:02, 2912.12rows/s]

Downloading:   0%|          | 168433/42438509 [01:07<4:02:16, 2907.85rows/s]

Downloading:   0%|          | 189231/42438509 [01:13<3:54:31, 3002.38rows/s]

Downloading:   0%|          | 209923/42438509 [01:20<3:48:48, 3075.96rows/s]

Downloading:   1%|          | 230682/42438509 [01:26<3:43:27, 3148.13rows/s]


ConnectionError: ('Connection aborted.', timeout('The write operation timed out'))

Error is from trying to send it back to Google big query. Don't want to run it again because it takes so long. Still was able to create csv file

In [10]:
All_data_sample_owners.to_csv('owners_sample.txt', sep= '\t')

In [ ]:
# Select * error
sql = """
    "SELECT * "
    "FROM `fleet-geode-327423.Wedge.transArchive_*` "
    "WHERE card_no IN (
    SELECT card_no 
    FROM `fleet-geode-327423.Wedge.Sample_Owner_list`
    )
"""
All_data_sample_owners = pandas_gbq.read_gbq(sql, project_id=gbq_proj_id)
pandas_gbq.to_gbq(All_data_sample_owners, 'Wedge.All_data_sample_owners', project_id=gbq_proj_id, if_exists='replace')

In [ ]:
table_ref = client.get_table('fleet-geode-327423.Wedge.All_data_sample_owners')

job_config = bigquery.QueryJobConfig(destination=table_ref)

sql = """
SELECT
  *
EXCEPT (tax,datetime)
FROM
  `fleet-geode-327423.Wedge.transArchive_*`
WHERE
  card_no IN (
  SELECT
    *
  FROM
    `fleet-geode-327423.Wedge.Sample_Owner_list`)
"""



query_job = client.query(sql, job_config=job_config)  # Make an API request.
query_job.result()  # Wait for the job to complete.

print("Query results loaded to the table {}".format(table_ref.table_id))